In [30]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [9]:
df = pd.read_excel("../Apoyo-Desafio/Telco-Customer-Churn.xlsx")

In [ ]:
# Definir las columnas categóricas
categorical_columns = ['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService',
					   'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
					   'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
					   'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn']

# Definir las columnas numéricas
numerical_columns = ['tenure', 'MonthlyCharges', 'TotalCharges']

# Generar estadísticas descriptivas para variables numéricas
numerical_stats = df.describe()
print("Estadísticas descriptivas para variables numéricas:")
print(numerical_stats)

# Generar estadísticas descriptivas para variables categóricas
categorical_stats = df[categorical_columns].describe()
print("\nEstadísticas descriptivas para variables categóricas:")
print(categorical_stats)

# Visualizar la distribución de las variables clave mediante histogramas
df.hist(bins=30, figsize=(20, 15))
plt.suptitle("Histogramas de variables numéricas")
plt.show()

# Visualizar la distribución de las variables clave mediante diagramas de caja
df.plot(kind='box', subplots=True, layout=(6, 4), figsize=(20, 20), sharex=False, sharey=False)
plt.suptitle("Diagramas de caja de variables numéricas")
plt.show()

# Análisis de correlaciones entre variables
correlation_matrix = df.corr()
print("\nMatriz de correlación:")
print(correlation_matrix)

# Visualizar la matriz de correlación mediante un mapa de calor
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title("Mapa de calor de la matriz de correlación")
plt.show()


# Resumen de hallazgos clave
print("\nConclusiones:")
print("1. Estadísticas descriptivas muestran la media, mediana, moda, desviación estándar y percentiles de las variables numéricas.")
print("2. Histogramas y diagramas de caja revelan la distribución de las variables y posibles valores atípicos.")
print("3. Gráficos de dispersión ayudan a identificar relaciones entre pares de variables.")
print("4. La matriz de correlación y el mapa de calor muestran las correlaciones entre variables.")
print("5. Valores atípicos identificados deben ser tratados según el contexto del análisis.")

#  Análisis Exploratorio de Datos (EDA)

El objetivo de esta sección es explorar las características del dataset, identificar patrones y detectar posibles problemas en los datos antes de entrenar los modelos de machine learning. 

## **1️ Definición de Variables**

Para facilitar el análisis, clasificamos las variables en dos grupos:

- **Variables Categóricas:** Incluyen información cualitativa sobre los clientes, como género, tipo de contrato y servicios contratados.
- **Variables Numéricas:** Contienen información cuantitativa, como duración de la suscripción, cargos mensuales y cargos totales.

---

## **2️ Análisis Estadístico de las Variables**
Antes de aplicar cualquier modelo, es fundamental entender la distribución de los datos mediante estadísticas descriptivas.

- **Variables numéricas:** Se analiza la media, mediana, desviación estándar y percentiles.
- **Variables categóricas:** Se revisa la cantidad de observaciones por cada categoría.

---

## **3️ Visualización de la Distribución de los Datos**
Para comprender la dispersión y la estructura de los datos, generamos diferentes visualizaciones:

###  **Histogramas**
Los histogramas permiten analizar la distribución de cada variable numérica y detectar sesgos en los datos.

###  **Diagramas de Caja (Boxplots)**
Los boxplots ayudan a identificar valores atípicos y visualizar la dispersión de los datos.

---

## **4️ Análisis de Correlación entre Variables**
Para evaluar relaciones entre variables y posibles colinealidades, generamos:

- **Matriz de correlación:** Muestra el coeficiente de correlación entre las variables numéricas.
- **Mapa de calor:** Representación visual de la matriz de correlación con `seaborn.heatmap`.

---

## **5️ Conclusiones del Análisis Exploratorio**
A partir de los gráficos y estadísticas obtenidos, podemos destacar los siguientes puntos clave:

1. **Las estadísticas descriptivas** nos permiten ver la media, mediana, moda y dispersión de los datos.
2. **Histogramas y diagramas de caja** ayudan a identificar valores extremos y patrones de distribución.
3. **Los gráficos de dispersión** pueden mostrar relaciones entre pares de variables relevantes.
4. **La matriz de correlación** indica posibles relaciones entre características.
5. **Valores atípicos** deben ser analizados y tratados adecuadamente antes del modelado.




In [ ]:
# Identificar valores nulos, duplicados y tipos de datos
print("Valores nulos por columna:")
print(df.isnull().sum())
print("\nValores duplicados:")
print(df.duplicated().sum())
print("\nTipos de datos:")
print(df.dtypes)

# Limpieza de datos
# Eliminar valores duplicados
df = df.drop_duplicates()

# Imputar valores nulos
df['TotalCharges'] = df['TotalCharges'].replace(" ", np.nan).astype(float)
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].mean())

# Transformación de datos
# Convertir variables categóricas en variables numéricas
label_encoders = {}
for column in categorical_columns:
    if df[column].dtype == 'object':
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
        label_encoders[column] = le

# Normalización y estandarización
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Informe de los pasos realizados
print("\nInforme de limpieza y transformación:")
print("1. Valores duplicados eliminados.")
print("2. Valores nulos imputados en la columna 'TotalCharges'.")
print("3. Variables categóricas convertidas en variables numéricas.")
print("4. Variables numéricas normalizadas usando Z-score Normalization.")

# Visualización de datos antes y después del preprocesamiento
print("\nDatos antes del preprocesamiento:")
print(df.head())

print("\nDatos después del preprocesamiento:")
print(df.head())


#  Preprocesamiento de Datos

En esta sección, me encargo de **limpiar y transformar** los datos para asegurar que estén en el formato adecuado antes de entrenar cualquier modelo de machine learning.

---

## **1️ Identificación de Problemas en los Datos**
Antes de aplicar transformaciones, lo primero que hago es revisar la calidad de los datos. Para ello, verifico:

- **Valores nulos**: Identifico si hay datos faltantes en alguna columna.
- **Valores duplicados**: Detecto si hay filas repetidas que podrían afectar el análisis.
- **Tipos de datos**: Me aseguro de que cada variable tenga el formato adecuado (números, categorías, etc.).

---

## **2️ Limpieza de Datos**
###  Eliminación de Duplicados  
Si encuentro filas duplicadas en el dataset, las elimino para evitar que influyan negativamente en el modelo.

###  Imputación de Valores Nulos  
En la columna `TotalCharges`, detecté algunos valores vacíos representados como `" "`. Lo que hago es:

1. Convertir esos valores en `NaN` para tratarlos correctamente.
2. Rellenar los valores `NaN` con la **media** de la columna, asegurando que no haya datos faltantes.

---

## **3️ Transformación de Datos**
###  Conversión de Variables Categóricas  
Dado que los modelos de machine learning no pueden manejar directamente variables categóricas, convierto estas variables en valores numéricos utilizando `LabelEncoder`.  

Cada categoría recibe un número único, lo que permite que los modelos interpreten la información correctamente.

---

## **4️ Normalización y Estandarización**
Para que los modelos trabajen mejor con las variables numéricas (`tenure`, `MonthlyCharges`, `TotalCharges`), aplico `StandardScaler`.  
Esto **normaliza** los valores usando la técnica **Z-score Normalization**, que centra los datos en media 0 y varianza 1.  

 **Ventaja:** Evita que una variable con valores muy grandes (ejemplo: `TotalCharges`) domine sobre otras con valores más pequeños.

---

## **5️ Resumen del Preprocesamiento**
Después de aplicar los pasos anteriores, genero un informe de los cambios realizados:

1. **Valores duplicados eliminados.**
2. **Valores nulos imputados en la columna `TotalCharges`.**
3. **Variables categóricas convertidas en valores numéricos.**
4. **Variables numéricas normalizadas usando `Z-score Normalization`.**

---

## **6️ Visualización de los Datos Antes y Después**
Finalmente, muestro un resumen del dataset antes y después del preprocesamiento para asegurarme de que las transformaciones se aplicaron correctamente.

Este proceso es clave para garantizar que los datos sean adecuados para el entrenamiento de modelos de machine learning y evitar sesgos que puedan afectar el rendimiento.


In [ ]:
# Definir las características (X) y la variable objetivo (y)
X = df.drop(columns=['Churn', 'customerID'])
y = df['Churn']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Tamaño del conjunto de entrenamiento: {X_train.shape[0]}')
print(f'Tamaño del conjunto de prueba: {X_test.shape[0]}')

In [ ]:

# Definir los modelos y sus hiperparámetros
models = {
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10]
        }
    },
    'GradientBoosting': {
        'model': GradientBoostingClassifier(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 7]
        }
    },
    'SVC': {
        'model': SVC(random_state=42),
        'params': {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf', 'poly'],
            'gamma': ['scale', 'auto']
        }
    }
}

# Entrenar y optimizar los modelos
best_models = {}
for name, model_info in models.items():
    grid_search = GridSearchCV(model_info['model'], model_info['params'], cv=5, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_models[name] = grid_search.best_estimator_
    print(f"Best parameters for {name}: {grid_search.best_params_}")
    print(f"Best score for {name}: {grid_search.best_score_}")

# Evaluar los mejores modelos en el conjunto de prueba
for name, model in best_models.items():
    y_pred = model.predict(X_test)
    print(f"Classification report for {name}:\n{classification_report(y_test, y_pred)}")

#  Entrenamiento y Optimización de Modelos de Machine Learning

En esta sección, me encargo de **definir, entrenar y optimizar** varios modelos de clasificación para predecir la fuga de clientes (*churn*). Utilizo **búsqueda de hiperparámetros** para obtener el mejor rendimiento posible.

---

## **1️ Selección de Modelos**
Para abordar este problema, seleccioné tres algoritmos de clasificación populares:

- **Random Forest:** Modelo basado en múltiples árboles de decisión, ideal para manejar datos con relaciones no lineales.
- **Gradient Boosting:** Algoritmo de boosting que ajusta errores progresivamente para mejorar la precisión.
- **SVM (Support Vector Machine):** Modelo basado en la maximización de márgenes entre clases.

Cada uno de estos modelos tiene ventajas específicas:
✅ **Random Forest** es robusto y maneja bien datos con muchas características.  
✅ **Gradient Boosting** suele lograr alta precisión optimizando el error en cada iteración.  
✅ **SVM** es útil cuando los datos no son linealmente separables y requiere pocos parámetros de ajuste.  

---

## **2️ Definición de Hiperparámetros**
Para cada modelo, defino una serie de hiperparámetros que quiero optimizar:

- **Random Forest:**
  - `n_estimators`: Número de árboles en el bosque.
  - `max_depth`: Profundidad máxima de cada árbol.
  - `min_samples_split`: Mínimo de muestras necesarias para dividir un nodo.

- **Gradient Boosting:**
  - `n_estimators`: Número de árboles a entrenar.
  - `learning_rate`: Controla cuánto cambia el modelo con cada iteración.
  - `max_depth`: Profundidad de los árboles en la fase de boosting.

- **SVC (Support Vector Machine):**
  - `C`: Parámetro de regularización (más alto, menos penalización).
  - `kernel`: Define la función del hiperplano (lineal, polinomial o radial).
  - `gamma`: Ajusta la influencia de los puntos de entrenamiento.

Estos hiperparámetros son clave porque afectan directamente el rendimiento del modelo.

---

## **3️ Entrenamiento y Optimización con GridSearchCV**
Para encontrar los mejores hiperparámetros, utilizo **GridSearchCV**, que prueba múltiples combinaciones y selecciona la mejor configuración.  

 **Ventaja de GridSearchCV:** Evalúa automáticamente cada combinación con validación cruzada (`cv=5`), lo que asegura que el modelo no esté sobreajustado.

El proceso consiste en:
1. Definir los modelos y sus hiperparámetros.
2. Aplicar `GridSearchCV` para buscar la mejor combinación.
3. Guardar el mejor modelo para cada algoritmo.

Después de entrenar cada modelo, imprimo los **mejores hiperparámetros** y la **mejor puntuación** obtenida durante la validación cruzada.

---

## **4️ Evaluación de los Modelos**
Una vez que tengo los modelos optimizados, los pruebo en el **conjunto de prueba (`X_test`)** para evaluar su rendimiento real.

Para esto, uso `classification_report`, que me proporciona métricas clave como:
- **Precisión (`precision`)**: Cuántos de los clientes que predije como "fugados" realmente lo son.
- **Sensibilidad (`recall`)**: Cuántos de los clientes fugados reales fueron detectados correctamente.
- **F1-score**: Un balance entre precisión y sensibilidad.
- **Exactitud (`accuracy`)**: Porcentaje total de predicciones correctas.

Cada modelo genera un informe con estas métricas, lo que me permite comparar su desempeño y elegir el más adecuado.

---

## **5️ Conclusión**
Después de evaluar los modelos, puedo determinar cuál es el mejor predictor para detectar clientes con riesgo de fuga.

 **Posibles mejoras futuras:**
- Evaluar otros modelos como **XGBoost** o **Redes Neuronales**.
- Usar técnicas de balanceo de clases si los datos están desbalanceados (`SMOTE`).
- Aplicar `RandomizedSearchCV` para acelerar la búsqueda de hiperparámetros.

Este proceso garantiza que obtenga el modelo más eficiente para ayudar a la empresa a reducir la fuga de clientes y mejorar la retención. 🚀🔥


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluar los mejores modelos en el conjunto de prueba
for name, model in best_models.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Metrics for {name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\n")